# **Detectron2 Fault detection on Solar Panel Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Installing Detectron2 on Colab**

In [ ]:
!pip install -q pyyaml==5.1
!pip install -q torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!pip install -q tqdm
!git clone -q https://github.com/facebookresearch/detectron2.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.6/763.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 74.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.8.0+cu101 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0+cu101 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 2.5 MB/s eta 0:00:00
     ━

In [ ]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2

In [ ]:
# unzipping dataset
# import zipfile
# with zipfile.ZipFile("/content/drive/MyDrive/Temp_files/panel_annots_2714_tts.zip","r") as zip_ref:
#    zip_ref.extractall("./")

## **Register the data to Detectron2 config**

In [ ]:
# !pip install -q roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="t0oCFO47DJrsXTDjba5d")
# project = rf.workspace("manish-dhruva-dbwyx").project("solar_panel_faulty")
# dataset = project.version(2).download("coco")

In [ ]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [ ]:
# function to create a dictionary of annotations and their objects
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        imgPath = img_anns["imagePath"].split('/')[-1]
        filename = os.path.join(directory, imgPath)
        
        record["file_name"] = filename
        record["height"] = 95
        record["width"] = 95
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']), #original
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
# Registering train and test dataset to detectron2 module
classes = ['faulty']

data_path = '/content/panel_annots_2714_tts/panel_annots_2714_tts/'

for d in ["train", "test"]:
    DatasetCatalog.register("category_" + d, lambda d=d: get_data_dicts(data_path+d, classes))
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_train")

In [ ]:
# Alternate method to register detectron2 using COCO.json format
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("category_train",{}, '/content/solar_panel_faulty/train/_annotations.coco.json', '/content/solar_panel_faulty/train')
# register_coco_instances("category_test",{},'/content/solar_panel_faulty/test/_annotations.coco.json', '/content/solar_panel_faulty/test')

# microcontroller_metadata = MetadataCatalog.get("category_train")
# dataset_train = DatasetCatalog.get(Data_Resister_training)
# dataset_valid = DatasetCatalog.get(Data_Resister_test)

 ## **Training the Detectron2 Instance Segmentation Model**

### **Model Building**

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.projects import point_rend
from tqdm import tqdm
import cv2
import time
from google.colab.patches import cv2_imshow
from PIL import Image

In [ ]:
# Object Detection model building
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/rpn_R_50_FPN_1x.yaml"))


cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/rpn_R_50_FPN_1x.yaml"))


cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 10000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### **Model Training** (Dont need to run if model weights are present)

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

[01/12 12:38:44 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:25, 6.56MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


In [ ]:
trainer.train()

In [ ]:
# # Saving the trained model weights 
from google.colab import files 
files.download("/content/output/model_final.pth")

## **Inference using the Trained Model**

In [ ]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Temp_files/output_10000/model_final.pth'
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Temp_files/model_final_10000i_2714p.pth'
# cfg.MODEL.WEIGHTS = '/content/model_final_02ce48.pkl'
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7
cfg.DATASETS.TEST = ("category_test", )
predictor = DefaultPredictor(cfg)

### Object Detection Model Inference

In [ ]:
# predicting on test data using instance segmentation

test_dataset_dicts = get_data_dicts(data_path+'test', classes)
for d in random.sample(test_dataset_dicts, 1):    
    img = cv2.imread(d["file_name"])
    
    starttime = time.time()
    outputs = predictor(img)
    print("time: ", time.time() - starttime)
    # vis = Visualizer(img[:, :, ::-1],
    #                metadata=microcontroller_metadata, 
    #                scale=2.0, 
    #                instance_mode=ColorMode.IMAGE_BW) # removes the colors of unsegmented pixels
    
    # # v = vis.draw_dataset_dict(d)
    # v = vis.draw_instance_predictions(outputs["instances"].to("cpu"))
    # plt.figure(figsize = (10, 5))
    # plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    # plt.show()

time:  0.10883355140686035


In [ ]:
# predicting on single test data

img2 = cv2.imread('/content/FNo442_33665.png')
starttime = time.time()
outputs = predictor(img2)
print("time: ",time.time()-starttime)
vis = Visualizer(img2[:, :, ::-1],
               metadata=microcontroller_metadata, 
               scale=2.0, 
               instance_mode=ColorMode.IMAGE_BW) # removes the colors of unsegmented pixels

# v = vis.draw_dataset_dict(d)
v = vis.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (10,5))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()